In [2]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [9]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv(
    Path("Resources/lending_data.csv")
)

# Review the DataFrame
lending_data_df.tail()

#Display DataFrame Shape
lending_data_df.shape

(77536, 8)

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [10]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df['loan_status']

# Separate the X variable, the features
X = lending_data_df.drop(columns=['loan_status'])

In [11]:
# Review the y variable Series
# y.info
y.tail()


77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, dtype: int64

In [12]:
# Review the X variable DataFrame
# X.info
X.tail()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300
77535,15600.0,9.742,72300,0.585062,9,2,42300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [14]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [16]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
# Due to 0 being significatly bigger number than 1 in the y Series, it helps to stratify the series
# to ensure the test and train data have the same proportion from the originakl dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [18]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data, use X_train and y_train data
logreg_model = logistic_regression_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [19]:
# Make a prediction using the testing data, use X_test data
test_predictions = logreg_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [20]:
# Print the balanced_accuracy score of the model
accurancy_score = balanced_accuracy_score(y_test, test_predictions)
print(f'The accurancy score of the model is ${accurancy_score}')

The accurancy score of the model is $0.9442676901753825


In [21]:
# Generate a confusion matrix for the model
# confusion_matrix(y_test, test_predictions)
logreg_matrix = confusion_matrix(y_test, test_predictions)
print("\t\tPredicted Positive\tPredicted Negative")
print("Actual Positive\t\t{}\t\t\t{}".format(logreg_matrix[1, 1], logreg_matrix[1, 0]))
print("Actual Negative\t\t{}\t\t\t{}".format(logreg_matrix[0, 1], logreg_matrix[0, 0]))

		Predicted Positive	Predicted Negative
Actual Positive		558			67
Actual Negative		80			18679


In [22]:
# Print the classification report for the model
print(classification_report(y_test, test_predictions,))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The 100% prediction of low-risk loans from the Logistic Regression model is indeed promising. However, the prediction for high-risk loans, while encouraging at 89%, would still warrant a cautious approach if I were a loan officer. It's essential to proceed with vigilance given the nature of high-risk predictions.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [24]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_over_sample = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_ros, y_ros = random_over_sample.fit_resample(X_train, y_train)

In [25]:
# Count the distinct values of the resampled labels data
y_ros.value_counts()

loan_status
0    56277
1    56277
Name: count, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [26]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# Use lbfgs solver
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier.fit(X_ros, y_ros)

# Make a prediction using the testing data
predictions_ros = classifier.predict(X_test)

# Check test data 
pd.DataFrame({"Predictions": predictions_ros, "Actual": y_test})

,Predictions,Actual
36831,0,0
75818,1,1
36563,0,0
13237,0,0
43292,0,0
...,...,...
38069,0,0
36892,0,0
5035,0,0
40821,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [27]:
# Print the balanced_accuracy score of the model 
predicted_ros_acc_score = balanced_accuracy_score(y_test, predictions_ros)
print(predicted_ros_acc_score)


0.9959744975744975


In [28]:
# Generate a confusion matrix for the model
predicted_ros_matrix = confusion_matrix(y_test, predictions_ros)
print("\t\tPredicted Positive\tPredicted Negative")
print("Actual Positive\t\t{}\t\t\t{}".format(predicted_ros_matrix[1, 1], predicted_ros_matrix[1, 0]))
print("Actual Negative\t\t{}\t\t\t{}".format(predicted_ros_matrix[0, 1], predicted_ros_matrix[0, 0]))

		Predicted Positive	Predicted Negative
Actual Positive		623			2
Actual Negative		91			18668


In [29]:
# Print the classification report for the model
print(classification_report(y_test,predictions_ros))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model, trained with oversampled data, demonstrates outstanding performance in predicting both healthy and high-risk loans. Achieving perfect recall (1.00) for both classes indicates that the model excels in identifying all true cases for each category. While the precision for high-risk loans is slightly lower at 0.87, implying some misclassification of healthy loans as high-risk, it remains notably high. Overall, the model exhibits a robust ability to differentiate between healthy and high-risk loans, making it a highly effective tool for this particular classification task.